In [160]:
# Load all the tools
import numpy as np
# from wisdem.aeroelasticse.CaseLibrary import ROSCO_Test
import os

from weis.aeroelasticse.FAST_reader import InputReader_Common, InputReader_OpenFAST, InputReader_FAST7
from weis.aeroelasticse.FAST_writer import InputWriter_Common, InputWriter_OpenFAST, InputWriter_FAST7
from weis.aeroelasticse.runFAST_pywrapper import runFAST_pywrapper_batch
from weis.aeroelasticse.CaseGen_General import CaseGen_General
from weis.aeroelasticse.CaseGen_IEC import CaseGen_IEC
from pCrunch import pdTools
from pCrunch import Processing, Analysis
from weis.aeroelasticse.Util import FileTools
import pandas as pd



In [161]:
caseFile = '/Users/dzalkind/Tools/WEIS/outputs/NASA/DLC_Play/case_matrix.yaml'

runDir   = os.path.dirname(caseFile)


# Later, make this multiple

In [162]:
# Load case matrix into dataframe
case_matrix = FileTools.load_yaml(caseFile, package=1)
cm = pd.DataFrame(case_matrix)

# Get ALL FAST info
fastFile = os.path.join(runDir,cm['Case_Name'][0] + '.fst')
FAST_ver = 'OpenFAST'
fastRead = InputReader_OpenFAST(FAST_ver=FAST_ver, dev_branch=True)
fastRead.FAST_InputFile = os.path.basename(fastFile)   # FAST input file (ext=.fst)
fastRead.FAST_directory = os.path.dirname(fastFile)   # Path to fst directory files
fastRead.execute()

# fastRead.fst_vt
# fastFile

# caseFile

Loading rotor performace data from text file: /Users/dzalkind/Tools/WEIS/outputs/NASA/DLC_Play/DLC_Test_000_Cp_Ct_Cq.txt


ValueError: could not convert string to float: '----------------------'

# Crunch the Data
Can skip if already run
TODO: use post_BatchRun() function

In [ ]:
# Set up pCrunch, run if not already
reCrunch = False

# Load and save statistics and load rankings
if not os.path.exists(os.path.join(runDir,'stats','dataset1_LoadRanking.yaml')) or reCrunch:
    # Initialize processing classes
    fp = Processing.FAST_Processing()

    # Set some processing parameters
    fp.OpenFAST_outfile_list    = [os.path.join(runDir, cn + '.out') for cn in cm.Case_Name]
    fp.t0                       = 100          # DZ: I'd like this to be 60 or 100, but there are errors there
    fp.parallel_analysis        = True
    fp.parallel_cores           = 8
    fp.results_dir              = os.path.join(runDir, 'stats')
    fp.verbose                  = True
    fp.save_LoadRanking         = True
    fp.save_SummaryStats        = True
    
    stats, load_rankings = fp.batch_processing()
    
else:
    stats = FileTools.load_yaml(os.path.join(runDir,'stats','dataset1_stats.yaml'))
    load_rankings = FileTools.load_yaml(os.path.join(runDir,'stats','dataset1_LoadRanking.yaml'))


Setup dataframes

In [163]:
if len(stats) == 1:
    st = pdTools.dict2df(stats[0])
    lr = pdTools.dict2df(load_rankings[0])
else:
    st = pdTools.dict2df(stats)
    lr = pdTools.dict2df(load_rankings)

# lr.columns

# len(stats)

# st.GenPwr['mean']

# st['meta']

# st.index = st['meta']['name']

# st['PtfmPitch']['max'].sort_values(ascending=False)

# cm['Case_Name']


# Power Production

In [164]:
if False:
    windspeeds, seed, IECtype, cm_wind = Processing.get_windspeeds(cm, return_df=True)

    pp = Analysis.Power_Production()
    Vavg = 10   # Average wind speed of cite
    Vrange = [2,26] # Range of wind speeds being considered
    indAEP = np.array(IECtype) == 'NTM'

    pp.windspeeds = np.sort(np.array(windspeeds)[indAEP])
    AEP = pp.AEP(st[indAEP],pp.windspeeds)
    RatedPower = fastRead.fst_vt['DISCON_in']['VS_RtPwr']
    print('Cap. Factor = {:3.3f}'.format(AEP/8760/RatedPower*1000))
    
    st['GenPwr']['mean'].sort_values(ascending=False)

# st

windspeeds, seed, IECtype, cm_wind = Processing.get_windspeeds(cm, return_df=True)

cm_wind.columns

Index([             ('IEC', 'DLC'),  ('AeroDyn15', 'AFAeroMod'),
         ('ElastoDyn', 'BlPitch1'),   ('ElastoDyn', 'BlPitch2'),
         ('ElastoDyn', 'BlPitch3'),     ('ElastoDyn', 'GenDOF'),
           ('ElastoDyn', 'NacYaw'),   ('ElastoDyn', 'RotSpeed'),
           ('ElastoDyn', 'YawDOF'),       ('Fst', 'OutFileFmt'),
                   ('Fst', 'TMax'),      ('HydroDyn', 'WaveHs'),
         ('HydroDyn', 'WaveSeed1'),      ('HydroDyn', 'WaveTp'),
        ('InflowWind', 'Filename'),  ('InflowWind', 'WindType'),
         ('ServoDyn', 'BlPitchF1'),   ('ServoDyn', 'BlPitchF2'),
         ('ServoDyn', 'BlPitchF3'),    ('ServoDyn', 'GenTiStp'),
            ('ServoDyn', 'PCMode'),  ('ServoDyn', 'PitManRat1'),
        ('ServoDyn', 'PitManRat2'),  ('ServoDyn', 'PitManRat3'),
         ('ServoDyn', 'TPitManS1'),   ('ServoDyn', 'TPitManS2'),
         ('ServoDyn', 'TPitManS3'),    ('ServoDyn', 'TimGenOf'),
                         'Case_ID',                 'Case_Name',
       ('InflowWind', 'Wi

# DELs
Look at damage equivalent loads

In [166]:
wc = pd.DataFrame(cm_wind[[('IEC','DLC'),(('InflowWind', 'WindSpeed'))]])

# st['TwrBsMyt']['DEL'].sort_values(ascending=False).head(10)

st_del = st['TwrBsMyt']['DEL']

# wc = cm_wind[[('IEC','DLC'),(('InflowWind', 'WindSpeed'))]]
# wc[('TwrBsMyt','DEL')] = st_del




st.index = st['meta']['name']

cm_wind.index = cm['Case_Name']

wc = cm_wind[[('IEC','DLC'),('InflowWind', 'WindSpeed'),('InflowWind', 'Seed')]]

st_red = pd.DataFrame()


st_red[('TwrBsMyt','DEL')] = st['TwrBsMyt']['DEL']
st_red[('TwrBsMyt','max')] = st['TwrBsMyt']['max']
st_red[('PtfmPitch','DEL')] = st['PtfmPitch']['DEL']
st_red[('PtfmHeave','DEL')] = st['PtfmHeave']['DEL']
st_red.index = st['meta']['name']




# res = pd.concat([wc,st_red])
res = wc.join(st_red)

# res.sort_values()

res.sort_values(('TwrBsMyt','DEL'),ascending=False).head(50)


# st_del.idxmax()
# st.sort_values([['TwrBsMyt']['DEL']],ascending=False).head(10)s


# st_del
# st_del
# wc = st['TwrBsMyt']['DEL']
# wc['PtfmPitch'] = st['PtfmPitch']['DEL']

# wc.columns()
# cm['Case_Name']

# st['PtfmHeave']['DEL'].sort_values(ascending=False).head(10)
# st['PtfmPitch']['DEL'].sort_values(ascending=False).head(10)
# st['RootMxb1']['DEL'].sort_values(ascending=False).head(10)

# worstDELs['RootMyb1']['DEL'] = st['RootMyb1']['DEL'].sort_values(ascending=False).head(10)

,"(IEC, DLC)","(InflowWind, WindSpeed)","(InflowWind, Seed)","(TwrBsMyt, DEL)","(TwrBsMyt, max)","(PtfmPitch, DEL)","(PtfmHeave, DEL)"
Case_Name,,,,,,,
DLC_Test_142,6.1,42.50,22.0,301714.165386,564765.314831,1.602421,4.073063
DLC_Test_143,6.1,42.50,22.0,300133.312728,566928.814051,1.648398,4.074169
DLC_Test_138,6.1,42.50,20.0,294040.433355,502373.764128,1.475596,3.755122
DLC_Test_139,6.1,42.50,20.0,289242.073700,475381.524815,1.511201,3.760913
DLC_Test_157,6.5,46.75,29.0,284169.486678,619541.471715,1.771813,4.154991
DLC_Test_156,6.5,46.75,29.0,283391.255265,629037.605018,1.762587,4.155312
DLC_Test_162,6.5,46.75,32.0,277689.395708,470682.168831,1.895012,3.620890
DLC_Test_163,6.5,46.75,32.0,275393.928486,442506.648384,1.886922,3.615641
DLC_Test_165,6.5,46.75,33.0,266981.409410,501647.065314,1.692968,3.666212


In [83]:
# # Compare to results in compRunDir
# # Check these maxima
# caseMeasures    = ['RotSpeed',
#                     'RootMyb',
#                     'TwrBsMyt',
#                     'PtfmPitch']

# loadRanksThis   = Processing.load_yaml(os.path.join(fp.results_dir,'dataset1_LoadRanking.yaml'))

# # If no comparison exists, just print own stats
# try:
#     loadRanksComp   = Processing.load_yaml(os.path.join(compRunDir,turbine,'stats','dataset1_LoadRanking.yaml'))

#     print('\t\t{}'.format(turbine))
#     print('----------------------------------------')
#     print('\t\t{}\t{}'.format('This Control','Baseline'))
#     for meas in caseMeasures:
#         try:
#             print('Max {} \t{:1.3e}\t{:1.3e}'.format(meas,loadRanksThis[meas]['max'][0],loadRanksComp[meas]['max'][0]))
#         except:
#             print('{} is not in stats'.format(meas))

# except:
#     print('No comparison files exist!')

#     print('\t\t{}'.format(turbine))
#     print('----------------------------------------')
#     print('\t\t{}\t{}'.format('This Control','Baseline'))
#     for meas in caseMeasures:
#         try:
#             print('Max {} \t{:1.3e}'.format(meas,loadRanksThis[meas]['max'][0]))
#         except:
#             print('{} is not in stats'.format(meas))
# windspeeds, seed, IECtype, cm_wind = Processing.get_windspeeds(cm, return_df=True)

# stats = FileTools.load_yaml(os.path.join(fp.results_dir,'dataset1_stats.yaml'))
# st = pdTools.dict2df(stats)

# pp = Analysis.Power_Production()
# Vavg = 10   # Average wind speed of cite
# Vrange = [2,26] # Range of wind speeds being considered
# indAEP = np.array(IECtype) == 'NTM'

# pp.windspeeds = np.sort(np.array(windspeeds)[indAEP])
# AEP = pp.AEP(st[indAEP],pp.windspeeds)
# RatedPower = fastRead.fst_vt['DISCON_in']['VS_RtPwr']
# print('Cap. Factor = {:3.3f}'.format(AEP/8760/RatedPower*1000))

# # Compare to results in compRunDir
# # Check these maxima
# caseMeasures    = ['RotSpeed',
#                     'RootMyb',
#                     'TwrBsMyt',
#                     'PtfmPitch']

# loadRanksThis   = Processing.load_yaml(os.path.join(fp.results_dir,'dataset1_LoadRanking.yaml'))

# # If no comparison exists, just print own stats
# try:
#     loadRanksComp   = Processing.load_yaml(os.path.join(compRunDir,turbine,'stats','dataset1_LoadRanking.yaml'))

#     print('\t\t{}'.format(turbine))
#     print('----------------------------------------')
#     print('\t\t{}\t{}'.format('This Control','Baseline'))
#     for meas in caseMeasures:
#         try:
#             print('Max {} \t{:1.3e}\t{:1.3e}'.format(meas,loadRanksThis[meas]['max'][0],loadRanksComp[meas]['max'][0]))
#         except:
#             print('{} is not in stats'.format(meas))

# except:
#     print('No comparison files exist!')

#     print('\t\t{}'.format(turbine))
#     print('----------------------------------------')
#     print('\t\t{}\t{}'.format('This Control','Baseline'))
#     for meas in caseMeasures:
#         try:
#             print('Max {} \t{:1.3e}'.format(meas,loadRanksThis[meas]['max'][0]))
#         except:
#             print('{} is not in stats'.format(meas))